In [162]:
import gurobipy as gp
from gurobipy import GRB
import json
import pandas as pd
import numpy as np

In [163]:
"Loading the Solar Production Data";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\DER_production.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Take the first row's hourly_profile_ratio list and make it a numpy array
Solar_CF = np.array(df["hourly_profile_ratio"][0])

In [164]:
Solar_CF

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.14, 0.21, 0.15, 0.12, 0.21,
       0.25, 0.85, 0.75, 0.55, 0.43, 0.23, 0.05, 0.25, 0.25, 0.  , 0.  ,
       0.  , 0.  ])

In [165]:
"Fullt Flexible Load (FFL) Data";
# Load JSON filen
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\usage_preference.json", "r") as f:
    data = json.load(f)

    data


In [166]:
min_FFL = data[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
min_FFL

8

In [167]:
"Electrcity Markert Data";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\bus_params.json", "r") as f:
    data = json.load(f)

electricty_price = data[0]["energy_price_DKK_per_kWh"]

In [168]:
price_import = data[0]["import_tariff_DKK/kWh"]
price_export = data[0]["export_tariff_DKK/kWh"]
#------------------------------------------
penalty_excess_import = data[0]["penalty_excess_import_DKK/kWh"]
penalty_excess_export = data[0]["penalty_excess_export_DKK/kWh"]
#-----------------------------------------------------
max_import = data[0]["max_import_kW"]
max_export = data[0]["max_export_kW"]

In [169]:
max_export

500

In [170]:
"Ramping Up for Solar and FFL";
with open(r"C:\Users\Pulin\Desktop\Optimisation_PowerSystems_Assignment1\data\question_1a\appliance_params.json", "r") as f:
    data = json.load(f)

max_power_solar = data["DER"][0]["max_power_kW"]
#min_power = data["DER"][0]["min_power_ratio"]
max_ramp_up_rate_solar = data["DER"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_solar = data["DER"][0]["max_ramp_rate_down_ratio"]

In [171]:
max_load_FFL = data["load"][0]["max_load_kWh_per_hour"]
min_load_FFL = data["load"][0]["min_load_ratio"]

max_ramp_up_rate_FFL= data["load"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_FFL= data["load"][0]["max_ramp_rate_down_ratio"]

In [172]:
max_load_FFL

3.0

- all units are in kW or KWh

In [173]:
t = range(len(Solar_CF))

In [174]:
max_load_FFL 

3.0

In [175]:
min_load_FFL

0.0

In [176]:
min_FFL

8

In [177]:

# Create model
model = gp.Model("Prosumer_Optimization")

# Decision variables
x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=GRB.INFINITY)
x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=GRB.INFINITY)
x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)  # scaled 0-1
z_import_excess = model.addVars(t, name="excess_import_kW", lb=0, ub=GRB.INFINITY)
z_export_excess = model.addVars(t, name="excess_export_kW", lb=0, ub=GRB.INFINITY)

# Hourly binary variables for tariffs
y_import = model.addVars(t, vtype=GRB.BINARY, name="tariff_import_activation")
y_export = model.addVars(t, vtype=GRB.BINARY, name="tariff_export_activation")

# Hourly constraints
for i in t:
    # FFL min/max
    model.addConstr(x_FFL[i] * max_load_FFL >= min_load_FFL, name=f"FFL_min_{i}")
    model.addConstr(x_FFL[i] * max_load_FFL <= max_load_FFL, name=f"FFL_max_{i}")

    # Energy balance: generation + imports - exports = load
    model.addConstr(
        x_imports[i] - x_exports[i] + Solar_CF[i] * max_power_solar == x_FFL[i] * max_load_FFL,
        name=f"energy_balance_{i}"
    )

    # Excess import/export
    model.addConstr(z_import_excess[i] >= x_imports[i] - max_import, name=f"excess_import_{i}")
    model.addConstr(z_export_excess[i] >= x_exports[i] - max_export, name=f"excess_export_{i}")

    # Binary tariff activation (big-M constraints)
    model.addConstr(x_imports[i] <= max_import * y_import[i], name=f"import_limit_{i}")
    model.addConstr(x_exports[i] <= max_export * y_export[i], name=f"export_limit_{i}")

    # Ramp constraints for FFL
    if i > 0:
        model.addConstr(
            (x_FFL[i] * max_load_FFL) - (x_FFL[i-1] * max_load_FFL) <= max_ramp_up_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_up_{i}"
        )
        model.addConstr(
            (x_FFL[i-1] * max_load_FFL) - (x_FFL[i] * max_load_FFL) <= max_ramp_down_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_down_{i}"
        )

# Minimum total daily FFL energy (at least 8 hours at full load)
model.addConstr(
    gp.quicksum(x_FFL[i] * max_load_FFL for i in t) >= min_FFL * max_load_FFL,
    name="FFL_min_total_energy"
)

# Objective: minimize cost + penalties
model.setObjective(
    gp.quicksum(
        price_import * x_imports[i] + penalty_excess_import * z_import_excess[i] +
        price_export * x_exports[i] + penalty_excess_export * z_export_excess[i]
        for i in t
    ),
    sense=GRB.MINIMIZE
)

# Solve
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 215 rows, 168 columns and 428 nonzeros
Model fingerprint: 0x9c1f6f49
Variable types: 120 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+03]
Found heuristic solution: objective 11.5560000
Presolve removed 203 rows and 145 columns
Presolve time: 0.01s
Presolved: 12 rows, 23 columns, 34 nonzeros
Variable types: 23 continuous, 0 integer (0 binary)

Root relaxation: objective 5.265000e+00, 6 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

In [178]:

# Total FFL consumption over 24 hours
total_FFL = sum(x_FFL[i].X * max_load_FFL for i in t)
print("Total FFL consumption over 24h: {:.2f} kWh".format(total_FFL))

Total FFL consumption over 24h: 24.00 kWh


In [179]:
# Print results
print("Optimal objective value:", model.ObjVal)
for i in t:
    print(f"Hour {i}:")
    print(f"  Imported power = {x_imports[i].X:.2f} kW")
    print(f"  Exported power = {x_exports[i].X:.2f} kW")
    print(f"  FFL consumption = {x_FFL[i].X*max_load_FFL:.2f} kW")
    print(f"  Excess import = {z_import_excess[i].X:.2f} kW")
    print(f"  Excess export = {z_export_excess[i].X:.2f} kW")
    print(f"  Import tariff active = {int(y_import[i].X)}")
    print(f"  Export tariff active = {int(y_export[i].X)}")

Optimal objective value: 5.265000000000001
Hour 0:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 1:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 2:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 3:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 4:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Ex

In [180]:
print("Optimal objective value:", model.ObjVal)
for i in t:
    print(f"Hour {i}:")
    print(f"  FFL consumption = {x_FFL[i].X*max_load_FFL:.2f} kW")
  

Optimal objective value: 5.265000000000001
Hour 0:
  FFL consumption = 0.00 kW
Hour 1:
  FFL consumption = 0.00 kW
Hour 2:
  FFL consumption = 0.00 kW
Hour 3:
  FFL consumption = 0.00 kW
Hour 4:
  FFL consumption = 0.00 kW
Hour 5:
  FFL consumption = 0.15 kW
Hour 6:
  FFL consumption = 3.00 kW
Hour 7:
  FFL consumption = 0.63 kW
Hour 8:
  FFL consumption = 3.00 kW
Hour 9:
  FFL consumption = 3.00 kW
Hour 10:
  FFL consumption = 0.63 kW
Hour 11:
  FFL consumption = 0.75 kW
Hour 12:
  FFL consumption = 3.00 kW
Hour 13:
  FFL consumption = 3.00 kW
Hour 14:
  FFL consumption = 3.00 kW
Hour 15:
  FFL consumption = 1.29 kW
Hour 16:
  FFL consumption = 0.90 kW
Hour 17:
  FFL consumption = 0.15 kW
Hour 18:
  FFL consumption = 0.75 kW
Hour 19:
  FFL consumption = 0.75 kW
Hour 20:
  FFL consumption = 0.00 kW
Hour 21:
  FFL consumption = 0.00 kW
Hour 22:
  FFL consumption = 0.00 kW
Hour 23:
  FFL consumption = 0.00 kW


In [181]:

# Total FFL consumption over 24 hours
total_FFL = sum(x_FFL[i].X * max_load_FFL for i in t)
print("Total FFL consumption over 24h: {:.2f} kWh".format(total_FFL))

Total FFL consumption over 24h: 24.00 kWh


## Without Using Binary Variables (use this):

- according to chatgpt binary variables don´t allow for dual variables

In [182]:

# Create model
model = gp.Model("Prosumer_Continuous")

# Decision variables
x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=max_import)
x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=max_export)
x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)  # scaled 0-1
z_import_excess = model.addVars(t, name="excess_import_kW", lb=0)
z_export_excess = model.addVars(t, name="excess_export_kW", lb=0)

# Hourly constraints
for i in t:
    # FFL min/max
    model.addConstr(x_FFL[i] * max_load_FFL >= min_load_FFL, name=f"FFL_min_{i}")
    model.addConstr(x_FFL[i] * max_load_FFL <= max_load_FFL, name=f"FFL_max_{i}")

    # Energy balance
    model.addConstr(
        x_imports[i] - x_exports[i] + Solar_CF[i] * max_power_solar == x_FFL[i] * max_load_FFL,
        name=f"energy_balance_{i}"
    )

    # Excess import/export
    model.addConstr(z_import_excess[i] >= x_imports[i] - max_import, name=f"excess_import_{i}")
    model.addConstr(z_export_excess[i] >= x_exports[i] - max_export, name=f"excess_export_{i}")

    # Ramp constraints for FFL
    if i > 0:
        model.addConstr(
            (x_FFL[i] * max_load_FFL) - (x_FFL[i-1] * max_load_FFL) <= max_ramp_up_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_up_{i}"
        )
        model.addConstr(
            (x_FFL[i-1] * max_load_FFL) - (x_FFL[i] * max_load_FFL) <= max_ramp_down_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_down_{i}"
        )

# Minimum total daily FFL energy
model.addConstr(
    gp.quicksum(x_FFL[i] * max_load_FFL for i in t) >= min_FFL * max_load_FFL,
    name="FFL_min_total_energy"
)

# Objective: minimize cost + penalties
model.setObjective(
    gp.quicksum(
        price_import * x_imports[i] + penalty_excess_import * z_import_excess[i] +
        price_export * x_exports[i] + penalty_excess_export * z_export_excess[i]
        for i in t
    ),
    sense=GRB.MINIMIZE
)

# Solve
model.optimize()

# Print results and duals
print("Optimal objective value:", model.ObjVal)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 167 rows, 120 columns and 332 nonzeros
Model fingerprint: 0xd9178a5d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [2e-01, 1e+03]
Presolve removed 151 rows and 74 columns
Presolve time: 0.01s
Presolved: 16 rows, 46 columns, 61 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.316250e+00   0.000000e+00      0s
       5    5.2650000e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.265000000e+00
Optimal objective value: 5.265000000000001


In [183]:
"Dual Variables";

print(model.pi)
print(" ")
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: dual = {constr.Pi}, slack = {constr.Slack}")

[0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5]
 
FFL_min_0: dual = 0.0, slack = -0.0
FFL_max_0: dual = 0.0, slack = 3.0
energy_balance_0: dual = 0.5, slack = 0.0
excess_import_0: dual = 0.0, slack = -1000.0
exce

In [184]:


total_FFL = sum(x_FFL[i].X * max_load_FFL for i in t)
print("Total FFL consumption over 24h: {:.2f} kWh".format(total_FFL))
print(" ")
# Optional: print hourly values
for i in t:
    print(f"Hour {i}: Imported = {x_imports[i].X:.2f}, Exported = {x_exports[i].X:.2f}, FFL = {x_FFL[i].X*max_load_FFL:.2f}")

Total FFL consumption over 24h: 24.00 kWh
 
Hour 0: Imported = 0.00, Exported = 0.00, FFL = 0.00
Hour 1: Imported = 0.00, Exported = 0.00, FFL = 0.00
Hour 2: Imported = 0.00, Exported = 0.00, FFL = 0.00
Hour 3: Imported = 0.00, Exported = 0.00, FFL = 0.00
Hour 4: Imported = 0.00, Exported = 0.00, FFL = 0.00
Hour 5: Imported = 2.85, Exported = 0.00, FFL = 3.00
Hour 6: Imported = 2.58, Exported = 0.00, FFL = 3.00
Hour 7: Imported = 2.37, Exported = 0.00, FFL = 3.00
Hour 8: Imported = 2.55, Exported = 0.00, FFL = 3.00
Hour 9: Imported = 0.18, Exported = 0.00, FFL = 0.54
Hour 10: Imported = 0.00, Exported = 0.00, FFL = 0.63
Hour 11: Imported = 0.00, Exported = 0.00, FFL = 0.75
Hour 12: Imported = 0.00, Exported = 0.00, FFL = 2.55
Hour 13: Imported = 0.00, Exported = 0.00, FFL = 2.25
Hour 14: Imported = 0.00, Exported = 0.00, FFL = 1.65
Hour 15: Imported = 0.00, Exported = 0.00, FFL = 1.29
Hour 16: Imported = 0.00, Exported = 0.00, FFL = 0.69
Hour 17: Imported = 0.00, Exported = 0.00, FFL =

### New Updated Model.

- solar can be curtailed fully and is limited by it´s PV potential CF:

In [217]:
import gurobipy as gp
from gurobipy import GRB

# Create model
model = gp.Model("Prosumer_Continuous")

# Decision variables
x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=max_import)
x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=max_export)
x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)  # scaled 0-1
x_solar = model.addVars(t, name="solar_power_kW", lb=0)  # new solar variable

z_import_excess = model.addVars(t, name="excess_import_kW", lb=0)
z_export_excess = model.addVars(t, name="excess_export_kW", lb=0)

# Hourly constraints
for i in t:
    # FFL min/max
    model.addConstr(x_FFL[i] * max_load_FFL >= min_load_FFL, name=f"FFL_min_{i}")
    model.addConstr(x_FFL[i] * max_load_FFL <= max_load_FFL, name=f"FFL_max_{i}")

    # Solar production limit
    model.addConstr(x_solar[i] <= Solar_CF[i] * max_power_solar, name=f"solar_max_{i}")

    # Energy balance
    model.addConstr(
        x_imports[i] - x_exports[i] + x_solar[i] == x_FFL[i] * max_load_FFL,
        name=f"energy_balance_{i}"
    )

    # Excess import/export
    model.addConstr(z_import_excess[i] >= x_imports[i] - max_import, name=f"excess_import_{i}")
    model.addConstr(z_export_excess[i] >= x_exports[i] - max_export, name=f"excess_export_{i}")

    # Ramp constraints for FFL + Solar,  [t - 1] -> [t]
    if i > 0:
        model.addConstr(
            (x_FFL[i] * max_load_FFL) - (x_FFL[i-1] * max_load_FFL) <= max_ramp_up_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_up_{i}"
        )
        model.addConstr(
            (x_FFL[i-1] * max_load_FFL) - (x_FFL[i] * max_load_FFL) <= max_ramp_down_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_down_{i}"
        )

        # Ramp constraints for solar
        model.addConstr(
            x_solar[i] - x_solar[i-1] <= max_ramp_up_rate_solar * max_power_solar,
            name=f"solar_ramp_up_{i}"
        )
        model.addConstr(
            x_solar[i-1] - x_solar[i] <= max_ramp_down_rate_solar * max_power_solar,
            name=f"solar_ramp_down_{i}"
        )

# Minimum total daily FFL energy
model.addConstr(
    gp.quicksum(x_FFL[i] * max_load_FFL for i in t) >= min_FFL * max_load_FFL,
    name="FFL_min_total_energy"
)

# Objective: minimize cost + penalties
model.setObjective(
    gp.quicksum(
        +(price_import + electricty_price[i])*x_imports[i] + penalty_excess_import*z_import_excess[i] +
        -(-price_export + electricty_price[i])*x_exports[i] + penalty_excess_export*z_export_excess[i]
        for i in t
    ),
    sense=GRB.MINIMIZE
)

# Solve
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 237 rows, 144 columns and 472 nonzeros
Model fingerprint: 0x9c4d2a1e
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [2e-01, 1e+03]
Presolve removed 221 rows and 90 columns
Presolve time: 0.01s
Presolved: 16 rows, 54 columns, 69 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.0800000e+03   4.373142e+03   0.000000e+00      0s
       5    1.4377500e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.437750000e+01


In [220]:
# Ensure the model was solved to optimality
if model.Status == GRB.OPTIMAL:
    total_consumption = sum(x_FFL[i].X * max_load_FFL for i in t)
    print(f"Total energy consumed by the prosumer over 24 hours: {total_consumption:.2f} kWh")
else:
    print("Model not solved to optimality. Status:", model.Status)


Total energy consumed by the prosumer over 24 hours: 24.00 kWh


In [219]:
if model.status == GRB.OPTIMAL:
    for i in t:
        print(f"Hour {i}:")
        print(f"  Imported power = {x_imports[i].X:.2f} kW")
        print(f"  Exported power = {x_exports[i].X:.2f} kW")
        print(f"  FFL consumption = {x_FFL[i].X * max_load_FFL:.2f} kW")
        print(f"  Excess import = {z_import_excess[i].X:.2f} kW")
        print(f"  Excess export = {z_export_excess[i].X:.2f} kW")
        print(f"  Solar power used = {x_solar[i].X:.2f} kW")
else:
    print("Model not solved to optimality. Status:", model.status)


Hour 0:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Solar power used = 0.00 kW
Hour 1:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Solar power used = 0.00 kW
Hour 2:
  Imported power = 3.00 kW
  Exported power = 0.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Solar power used = 0.00 kW
Hour 3:
  Imported power = 3.00 kW
  Exported power = 0.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Solar power used = 0.00 kW
Hour 4:
  Imported power = 3.00 kW
  Exported power = 0.00 kW
  FFL consumption = 3.00 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Solar power used = 0.00 kW
Hour 5:
  Imported power = 0.00 kW
  Exported power = 0.00 kW
  FFL consumption = 0.15 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  S